## Screening Technical Analysis for Crypto on two Timeframes

We'll explore a series of indicators separated by category on two different timeframes

In [ ]:
import ccxt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ta.volatility import BollingerBands
from ta.volatility import AverageTrueRange

from ta.trend import MACD
from ta.trend import AroonIndicator
from ta.trend import IchimokuIndicator
from ta.trend import ADXIndicator
from ta.trend import PSARIndicator

from ta.momentum import RSIIndicator
from ta.momentum import StochasticOscillator

from ta.volume import VolumeWeightedAveragePrice
from ta.volume import OnBalanceVolumeIndicator
from ta.volume import AccDistIndexIndicator
from ta.volume import ChaikinMoneyFlowIndicator

from IPython.display import display
from tabulate import tabulate
from datetime import datetime


# Set the style of matplotlib to 'dark_background'
plt.style.use('dark_background')

# Configure pandas to display all rows and columns without line breaks
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)  # Set display width to 0 to prevent wrapping
pd.set_option('display.max_colwidth', None)  # Deprecated, use None instead


In [ ]:
universe = [
    'BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'NEOUSDT', 'LTCUSDT', 'QTUMUSDT', 'ADAUSDT', 'XRPUSDT', 'EOSUSDT', 'IOTAUSDT',
    'XLMUSDT', 'ONTUSDT', 'TRXUSDT', 'ETCUSDT', 'ICXUSDT', 'VETUSDT', 'LINKUSDT', 'WAVESUSDT', 'HOTUSDT',
    'ZILUSDT', 'ZRXUSDT', 'FETUSDT', 'BATUSDT', 'XMRUSDT', 'ZECUSDT', 'IOSTUSDT', 'CELRUSDT', 'DASHUSDT', 'OMGUSDT',
    'THETAUSDT', 'ENJUSDT', 'MATICUSDT', 'ATOMUSDT', 'ONEUSDT', 'FTMUSDT', 'ALGOUSDT', 'DOGEUSDT', 'DUSKUSDT', 'ANKRUSDT',
    'COCOSUSDT', 'MTLUSDT', 'TOMOUSDT', 'DENTUSDT', 'KEYUSDT', 'CVCUSDT', 'CHZUSDT', 'BANDUSDT', 'XTZUSDT', 'RENUSDT',
    'RVNUSDT', 'HBARUSDT', 'NKNUSDT', 'STXUSDT', 'KAVAUSDT', 'ARPAUSDT', 'IOTXUSDT', 'RLCUSDT', 'BCHUSDT', 'FTTUSDT',
    'OGNUSDT', 'BTSUSDT', 'BNTUSDT', 'COTIUSDT', 'SOLUSDT', 'CTSIUSDT', 'CHRUSDT', 'MDTUSDT', 'STMXUSDT', 'KNCUSDT',
    'LRCUSDT', 'COMPUSDT', 'SCUSDT', 'ZENUSDT', 'SNXUSDT', 'DGBUSDT', 'SXPUSDT', 'MKRUSDT', 'STORJUSDT', 'MANAUSDT',
    'YFIUSDT', 'BALUSDT', 'BLZUSDT', 'SRMUSDT', 'ANTUSDT', 'CRVUSDT', 'SANDUSDT', 'OCEANUSDT', 'NMRUSDT', 'DOTUSDT',
    'RSRUSDT', 'TRBUSDT', 'SUSHIUSDT', 'KSMUSDT', 'EGLDUSDT', 'RUNEUSDT', 'UMAUSDT', 'BELUSDT', 'UNIUSDT', 'OXTUSDT',
    'AVAXUSDT', 'HNTUSDT', 'FLMUSDT', 'XVSUSDT', 'ALPHAUSDT', 'AAVEUSDT', 'NEARUSDT', 'FILUSDT', 'INJUSDT', 'AUDIOUSDT',
    'CTKUSDT', 'AXSUSDT', 'UNFIUSDT', 'ROSEUSDT', 'XEMUSDT', 'SKLUSDT', 'GRTUSDT', '1INCHUSDT', 'REEFUSDT', 'CELOUSDT',
    'TRUUSDT', 'CKBUSDT', 'LITUSDT', 'SFPUSDT', 'ALICEUSDT', 'LINAUSDT', 'PERPUSDT', 'CFXUSDT', 'TLMUSDT', 'BAKEUSDT',
    'ICPUSDT', 'ARUSDT', 'MASKUSDT', 'LPTUSDT', 'XVGUSDT', 'ATAUSDT', 'GTCUSDT', 'KLAYUSDT', 'C98USDT', 'QNTUSDT',
    'FLOWUSDT', 'MINAUSDT', 'RAYUSDT', 'DYDXUSDT', 'IDEXUSDT', 'GALAUSDT', 'YGGUSDT', 'AGLDUSDT', 'RADUSDT', 'DARUSDT',
    'BNXUSDT', 'ENSUSDT', 'JASMYUSDT', 'RNDRUSDT', 'FXSUSDT', 'HIGHUSDT', 'CVXUSDT', 'PEOPLEUSDT', 'SPELLUSDT', 'JOEUSDT',
    'ACHUSDT', 'IMXUSDT', 'API3USDT', 'WOOUSDT', 'TUSDT', 'ASTRUSDT', 'GMTUSDT', 'APEUSDT', 'GALUSDT', 'LDOUSDT',
    'OPUSDT', 'LEVERUSDT', 'STGUSDT', 'GMXUSDT', 'APTUSDT', 'HFTUSDT'
]

### Fetching OHLCV for the two Timeframes

In [ ]:
exchange = ccxt.binance({
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future',
    }
})

def fetch_ohlcv_data(symbol, timeframe):
    ohlcv_data = exchange.fetch_ohlcv(symbol, timeframe)
    ticker_info = exchange.fetchTicker(symbol)
    vwap = ticker_info.get('vwap')  
    pct_change = ticker_info.get('percentage')
    for i in range(len(ohlcv_data)):
        # Use datetime.utcfromtimestamp correctly
        ohlcv_data[i][0] = datetime.utcfromtimestamp(ohlcv_data[i][0] / 1000)
        ohlcv_data[i].append(vwap)
        ohlcv_data[i].append(pct_change)  
    return ohlcv_data

ohlcv_daily = {}
ohlcv_long = {}
ohlcv_short = {}

# Adjust timeframes here
long_timeframe = '1h'
short_timeframe = '15m'

for symbol in universe:
    daily_data = fetch_ohlcv_data(symbol, '1d')
    ohlcv_daily[symbol] = pd.DataFrame(daily_data, columns=['date', 'open', 'high', 'low', 'close', 'volume', 'vwap', 'pct_change'])
    
    long_data = fetch_ohlcv_data(symbol, long_timeframe)
    ohlcv_long[symbol] = pd.DataFrame(long_data, columns=['date', 'open', 'high', 'low', 'close', 'volume', 'vwap', 'pct_change'])
    
    short_data = fetch_ohlcv_data(symbol, short_timeframe)
    ohlcv_short[symbol] = pd.DataFrame(short_data, columns=['date', 'open', 'high', 'low', 'close', 'volume', 'vwap', 'pct_change'])


--- 

## Technical Analysis (Trend, Momentum, Volatility, Volume)

### Technical Analysis on Daily

In [ ]:
technical_daily_df = pd.DataFrame()

for symbol, df in ohlcv_daily.items():

    pct_change = df['pct_change'].iloc[-1]
    close = df['close'].iloc[-1]
    vwap = df['vwap'].iloc[-1]
    if vwap == None:
        continue
    vwap_position = -1 if vwap < close else 1

    # Simple Moving Average Crosses
    df['20_day_sma'] = df['close'].rolling(window=20).mean()
    df['50_day_sma'] = df['close'].rolling(window=50).mean()
    df['ma_cross'] = 0  # Default value for no cross
    bullish_cross = (df['20_day_sma'] > df['50_day_sma']) & (df['20_day_sma'].shift(1) <= df['50_day_sma'].shift(1))
    df.loc[bullish_cross, 'ma_cross'] = 1
    bearish_cross = (df['20_day_sma'] < df['50_day_sma']) & (df['20_day_sma'].shift(1) >= df['50_day_sma'].shift(1))
    df.loc[bearish_cross, 'ma_cross'] = -1
    
    # MACD
    macd_indicator = MACD(df['close'])
    df['macd'] = macd_indicator.macd()
    df['macd_signal'] = macd_indicator.macd_signal()
    df['macd_diff'] = macd_indicator.macd_diff()
    df['macd_cross'] = 0 
    macd_bullish_cross = (df['macd'] > df['macd_signal']) & (df['macd'].shift(1) <= df['macd_signal'].shift(1))
    df.loc[macd_bullish_cross, 'macd_cross'] = 1
    macd_bearish_cross = (df['macd'] < df['macd_signal']) & (df['macd'].shift(1) >= df['macd_signal'].shift(1))
    df.loc[macd_bearish_cross, 'macd_cross'] = -1

    # Aroon Indicator
    aroon_indicator = AroonIndicator(high=df['high'], low=df['low'], window=25)
    df['aroon_up'] = aroon_indicator.aroon_up()
    df['aroon_down'] = aroon_indicator.aroon_down()
    df['aroon_cross'] = 0
    df['aroon_position'] = 0
    aroon_position_condition_1 = (df['aroon_up'] > 80) & (df['aroon_down'] < 20)
    df.loc[aroon_position_condition_1, 'aroon_position'] = 1
    aroon_position_condition_2 = (df['aroon_down'] > 80) & (df['aroon_up'] < 20)
    df.loc[aroon_position_condition_2, 'aroon_position'] = -1
    aroon_bull_cross = (df['aroon_up'] > df['aroon_down']) & (df['aroon_up'].shift(1) <= df['aroon_down'].shift(1))
    df.loc[aroon_bull_cross, 'aroon_cross'] = 1
    aroon_bear_cross = (df['aroon_up'] < df['aroon_down']) & (df['aroon_up'].shift(1) >= df['aroon_down'].shift(1))
    df.loc[aroon_bear_cross, 'aroon_cross'] = -1

    # ADX
    adx_indicator = ADXIndicator(high=df['high'], low=df['low'], close=df['close'], window=14, fillna=False)
    df['adx'] = adx_indicator.adx()

    # Parabolic (SAR)
    psar_indicator = PSARIndicator(df['high'], df['low'], df['close'], step=0.02, max_step=0.2)
    df['psar'] = psar_indicator.psar()
    df['psar_position'] = 0
    df.loc[df['psar'] > df['close'], 'psar_position'] = -1
    df.loc[df['psar'] < df['close'], 'psar_position'] = 1
    # When there is a change in the dot. Above to below is bullish (1), below to above is bearish (-1), and 0 otherwise
    df['psar_change'] = 0  # Default value for no change
    psar_above_to_below = (df['psar_position'] > 0) & (df['psar_position'].shift(1) <= 0)
    psar_below_to_above = (df['psar_position'] < 0) & (df['psar_position'].shift(1) >= 0)
    df.loc[psar_above_to_below, 'psar_change'] = 1  # Bullish change from above to below
    df.loc[psar_below_to_above, 'psar_change'] = -1  # Bearish change from below to above

    # Ichimoku Cloud
    ichimoku = IchimokuIndicator(high=df['high'], low=df['low'], window1=9, window2=26, window3=52, visual=False)
    df['conversion_line'] = ichimoku.ichimoku_conversion_line()
    df['base_line'] = ichimoku.ichimoku_base_line()
    df['leading_span_a'] = ichimoku.ichimoku_a()
    df['leading_span_b'] = ichimoku.ichimoku_b()
    # Identify bullish and bearish crosses
    bullish_cross = (df['conversion_line'] > df['base_line']) & (df['conversion_line'].shift(1) <= df['base_line'].shift(1))
    bearish_cross = (df['conversion_line'] < df['base_line']) & (df['conversion_line'].shift(1) >= df['base_line'].shift(1))
    df['ichimoku_cross'] = 0
    df.loc[bullish_cross, 'ichimoku_cross'] = 1
    df.loc[bearish_cross, 'ichimoku_cross'] = -1
    # Determine the position of the price relative to the cloud
    above_cloud = (df['close'] > df[['leading_span_a', 'leading_span_b']].max(axis=1))
    below_cloud = (df['close'] < df[['leading_span_a', 'leading_span_b']].min(axis=1))
    # Corrected logic for determining if the price is within the cloud
    in_cloud = ~above_cloud & ~below_cloud
    df['cloud_position'] = 0
    df.loc[above_cloud, 'cloud_position'] = 1
    df.loc[below_cloud, 'cloud_position'] = -1
    df.loc[in_cloud, 'cloud_position'] = 0

    # RSI
    rsi_period = 14 
    rsi_indicator = RSIIndicator(df['close'], rsi_period)
    df['rsi'] = rsi_indicator.rsi()
    rsi_value = df['rsi'].iloc[-1]
    rsi_signal = 1 if rsi_value < 30 else -1 if rsi_value > 70 else 0

    # Stochastic Oscillator
    stoch_window = 14
    stoch_smooth_window = 3
    stoch = StochasticOscillator(high=df['high'], low=df['low'], close=df['close'], window=stoch_window, smooth_window=stoch_smooth_window)
    df['%K'] = stoch.stoch()
    df['%D'] = stoch.stoch_signal()
    stoch_signal = 1 if df['%K'].iloc[-1] < 20 else -1 if df['%K'].iloc[-1] > 80 else 0

    # Average True Range
    atr_indicator = AverageTrueRange(df['high'], df['low'], df['close'], window=14)
    df['atr'] = atr_indicator.average_true_range()

    # Bollinger Bands
    indicator_bb = BollingerBands(close=df['close'], window=20, window_dev=2)
    df['bb_mavg'], df['bb_hband'], df['bb_lband'] = indicator_bb.bollinger_mavg(), indicator_bb.bollinger_hband(), indicator_bb.bollinger_lband()
    df['bb_position'] = 0
    df['bb_position'] = np.where(df['close'] > df['bb_hband'], -1, np.where(df['close'] < df['bb_lband'], 1, 0))

    # Calculate On Balance Volume (OBV)
    obv_indicator = OnBalanceVolumeIndicator(df['close'], df['volume'])
    df['obv'] = obv_indicator.on_balance_volume()
    obv_trend = df['obv'].diff(periods=5)
    obv_signal = 1 if obv_trend.iloc[-1] > 0 else -1

    adi_indicator = AccDistIndexIndicator(high=df['high'], low=df['low'], close=df['close'], volume=df['volume'])
    df['adi'] = adi_indicator.acc_dist_index()
    adi_trend = df['adi'].diff(periods=5)
    adi_signal = 1 if adi_trend.iloc[-1] > 0 else -1
    
    # Calculate Chaikin Money Flow (CMF)
    df['cmf'] = ChaikinMoneyFlowIndicator(high=df['high'], low=df['low'], close=df['close'], volume=df['volume'], window=28, fillna=False).chaikin_money_flow()
    cmf_value = df['cmf'].iloc[-1]
    cmf_position = 1 if cmf_value > 0 else (-1 if cmf_value < 0 else 0)
    previous_cmf_value = df['cmf'].shift(1).iloc[-1]
    cmf_cross = 1 if (cmf_value > 1 and previous_cmf_value < -1) else (-1 if (cmf_value < -1 and previous_cmf_value > 1) else 0)
        
    # Append the result to the momentum_df DataFrame using concat instead of append
    technical_daily_df = pd.concat([technical_daily_df, pd.DataFrame({
        'symbol': [symbol],
        'close': [df['close'].iloc[-1]],
        'vwap': [vwap],
        'pct_change': [pct_change],
        'vwap_position': [vwap_position],
        '20_50_day_ma_cross': [df['ma_cross'].iloc[-1]],
        'macd_cross': [df['macd_cross'].iloc[-1]],
        'aroon_position': [df['aroon_position'].iloc[-1]],
        'aroon_cross': [df['aroon_cross'].iloc[-1]],
        'adx': [df['adx'].iloc[-1]],
        'psar_position': [df['psar_position'].iloc[-1]],
        'psar_change': [df['psar_change'].iloc[-1]],
        'cloud_position': [df['cloud_position'].iloc[-1]],
        'ichimoku_cross': [df['ichimoku_cross'].iloc[-1]],
        'rsi': [rsi_signal],  
        'stoch': [stoch_signal],
        'atr': [df['atr'].iloc[-1]],
        'bb_position': [df['bb_position'].iloc[-1]],
        'obv_signal': [obv_signal],
        'adi_signal': [adi_signal],
        'cmf_position': [cmf_position],
        'cmf_cross': [cmf_cross],
    })], ignore_index=True)

# Order momentum_df by 'adx' from highest to lowest
technical_daily_df.sort_values(by='adx', ascending=False, inplace=True)

# Set the 'symbol' column as the index of momentum_df
technical_daily_df.set_index('symbol', inplace=True)

# display(trend_long_df)

### Technical Report Daily

In [ ]:
bullish_counts_daily = {}
bearish_counts_daily = {}

# Loop through each row in the DataFrame
for index, row in technical_daily_df.iterrows():
    bullish_count = 0
    bearish_count = 0
    bullish_signals = []
    bearish_signals = []
    
    # Loop through each column to count the bullish and bearish signals
    for col in technical_daily_df.columns:
        if row[col] == 1:
            bullish_count += 1
            bullish_signals.append(col)
        elif row[col] == -1:
            bearish_count += 1
            bearish_signals.append(col)
    
    # Update the counts in the dictionaries
    bullish_counts_daily[index] = {
        'bullish_count': bullish_count,
        'bearish_count': bearish_count,
        'bullish_signals': bullish_signals,
        'bearish_signals': bearish_signals,
        'adx': row['adx'],
        'atr': row['atr'],
        'close': row['close'],
        'vwap': row['vwap'],
        'pct_change': row['pct_change']
    }
    
    bearish_counts_daily[index] = {
        'bearish_count': bearish_count,
        'bullish_count': bullish_count,
        'bullish_signals': bullish_signals,
        'bearish_signals': bearish_signals,
        'adx': row['adx'],
        'atr': row['atr'],
        'close': row['close'],
        'vwap': row['vwap'],
        'pct_change': row['pct_change']
    }

# Filter the dictionaries to include only items with at least  bullish and bearish counts
filtered_bullish_daily = {k: v for k, v in bullish_counts_daily.items() if v['bullish_count'] >= 5}
filtered_bearish_daily = {k: v for k, v in bearish_counts_daily.items() if v['bearish_count'] >= 5}

# Convert to DataFrame for better visualization
df_filtered_bullish_daily = pd.DataFrame.from_dict(filtered_bullish_daily, orient='index')
df_filtered_bearish_daily = pd.DataFrame.from_dict(filtered_bearish_daily, orient='index')


### Technical Analysis Long Timeframe

In [ ]:
technical_long_df = pd.DataFrame()

for symbol, df in ohlcv_long.items():

    pct_change = df['pct_change'].iloc[-1]
    close = df['close'].iloc[-1]
    vwap = df['vwap'].iloc[-1]
    if vwap == None:
        continue
    vwap_position = -1 if vwap < close else 1

    # Simple Moving Average Crosses
    df['20_day_sma'] = df['close'].rolling(window=20).mean()
    df['50_day_sma'] = df['close'].rolling(window=50).mean()
    df['ma_cross'] = 0  # Default value for no cross
    bullish_cross = (df['20_day_sma'] > df['50_day_sma']) & (df['20_day_sma'].shift(1) <= df['50_day_sma'].shift(1))
    df.loc[bullish_cross, 'ma_cross'] = 1
    bearish_cross = (df['20_day_sma'] < df['50_day_sma']) & (df['20_day_sma'].shift(1) >= df['50_day_sma'].shift(1))
    df.loc[bearish_cross, 'ma_cross'] = -1
    
    # MACD
    macd_indicator = MACD(df['close'])
    df['macd'] = macd_indicator.macd()
    df['macd_signal'] = macd_indicator.macd_signal()
    df['macd_diff'] = macd_indicator.macd_diff()
    df['macd_cross'] = 0 
    macd_bullish_cross = (df['macd'] > df['macd_signal']) & (df['macd'].shift(1) <= df['macd_signal'].shift(1))
    df.loc[macd_bullish_cross, 'macd_cross'] = 1
    macd_bearish_cross = (df['macd'] < df['macd_signal']) & (df['macd'].shift(1) >= df['macd_signal'].shift(1))
    df.loc[macd_bearish_cross, 'macd_cross'] = -1

    # Aroon Indicator
    aroon_indicator = AroonIndicator(high=df['high'], low=df['low'], window=25)
    df['aroon_up'] = aroon_indicator.aroon_up()
    df['aroon_down'] = aroon_indicator.aroon_down()
    df['aroon_cross'] = 0
    df['aroon_position'] = 0
    aroon_position_condition_1 = (df['aroon_up'] > 80) & (df['aroon_down'] < 20)
    df.loc[aroon_position_condition_1, 'aroon_position'] = 1
    aroon_position_condition_2 = (df['aroon_down'] > 80) & (df['aroon_up'] < 20)
    df.loc[aroon_position_condition_2, 'aroon_position'] = -1
    aroon_bull_cross = (df['aroon_up'] > df['aroon_down']) & (df['aroon_up'].shift(1) <= df['aroon_down'].shift(1))
    df.loc[aroon_bull_cross, 'aroon_cross'] = 1
    aroon_bear_cross = (df['aroon_up'] < df['aroon_down']) & (df['aroon_up'].shift(1) >= df['aroon_down'].shift(1))
    df.loc[aroon_bear_cross, 'aroon_cross'] = -1

    # ADX
    adx_indicator = ADXIndicator(high=df['high'], low=df['low'], close=df['close'], window=14, fillna=False)
    df['adx'] = adx_indicator.adx()

    # Parabolic (SAR)
    psar_indicator = PSARIndicator(df['high'], df['low'], df['close'], step=0.02, max_step=0.2)
    df['psar'] = psar_indicator.psar()
    df['psar_position'] = 0
    df.loc[df['psar'] > df['close'], 'psar_position'] = -1
    df.loc[df['psar'] < df['close'], 'psar_position'] = 1
    # When there is a change in the dot. Above to below is bullish (1), below to above is bearish (-1), and 0 otherwise
    df['psar_change'] = 0  # Default value for no change
    psar_above_to_below = (df['psar_position'] > 0) & (df['psar_position'].shift(1) <= 0)
    psar_below_to_above = (df['psar_position'] < 0) & (df['psar_position'].shift(1) >= 0)
    df.loc[psar_above_to_below, 'psar_change'] = 1  # Bullish change from above to below
    df.loc[psar_below_to_above, 'psar_change'] = -1  # Bearish change from below to above

    # Ichimoku Cloud
    ichimoku = IchimokuIndicator(high=df['high'], low=df['low'], window1=9, window2=26, window3=52, visual=False)
    df['conversion_line'] = ichimoku.ichimoku_conversion_line()
    df['base_line'] = ichimoku.ichimoku_base_line()
    df['leading_span_a'] = ichimoku.ichimoku_a()
    df['leading_span_b'] = ichimoku.ichimoku_b()
    # Identify bullish and bearish crosses
    bullish_cross = (df['conversion_line'] > df['base_line']) & (df['conversion_line'].shift(1) <= df['base_line'].shift(1))
    bearish_cross = (df['conversion_line'] < df['base_line']) & (df['conversion_line'].shift(1) >= df['base_line'].shift(1))
    df['ichimoku_cross'] = 0
    df.loc[bullish_cross, 'ichimoku_cross'] = 1
    df.loc[bearish_cross, 'ichimoku_cross'] = -1
    # Determine the position of the price relative to the cloud
    above_cloud = (df['close'] > df[['leading_span_a', 'leading_span_b']].max(axis=1))
    below_cloud = (df['close'] < df[['leading_span_a', 'leading_span_b']].min(axis=1))
    # Corrected logic for determining if the price is within the cloud
    in_cloud = ~above_cloud & ~below_cloud
    df['cloud_position'] = 0
    df.loc[above_cloud, 'cloud_position'] = 1
    df.loc[below_cloud, 'cloud_position'] = -1
    df.loc[in_cloud, 'cloud_position'] = 0

    # RSI
    rsi_period = 14 
    rsi_indicator = RSIIndicator(df['close'], rsi_period)
    df['rsi'] = rsi_indicator.rsi()
    rsi_value = df['rsi'].iloc[-1]
    rsi_signal = 1 if rsi_value < 30 else -1 if rsi_value > 70 else 0

    # Stochastic Oscillator
    stoch_window = 14
    stoch_smooth_window = 3
    stoch = StochasticOscillator(high=df['high'], low=df['low'], close=df['close'], window=stoch_window, smooth_window=stoch_smooth_window)
    df['%K'] = stoch.stoch()
    df['%D'] = stoch.stoch_signal()
    stoch_signal = 1 if df['%K'].iloc[-1] < 20 else -1 if df['%K'].iloc[-1] > 80 else 0

    # Average True Range
    atr_indicator = AverageTrueRange(df['high'], df['low'], df['close'], window=14)
    df['atr'] = atr_indicator.average_true_range()

    # Bollinger Bands
    indicator_bb = BollingerBands(close=df['close'], window=20, window_dev=2)
    df['bb_mavg'], df['bb_hband'], df['bb_lband'] = indicator_bb.bollinger_mavg(), indicator_bb.bollinger_hband(), indicator_bb.bollinger_lband()
    df['bb_position'] = 0
    df['bb_position'] = np.where(df['close'] > df['bb_hband'], -1, np.where(df['close'] < df['bb_lband'], 1, 0))

    # Calculate On Balance Volume (OBV)
    obv_indicator = OnBalanceVolumeIndicator(df['close'], df['volume'])
    df['obv'] = obv_indicator.on_balance_volume()
    obv_trend = df['obv'].diff(periods=5)
    obv_signal = 1 if obv_trend.iloc[-1] > 0 else -1

    adi_indicator = AccDistIndexIndicator(high=df['high'], low=df['low'], close=df['close'], volume=df['volume'])
    df['adi'] = adi_indicator.acc_dist_index()
    adi_trend = df['adi'].diff(periods=5)
    adi_signal = 1 if adi_trend.iloc[-1] > 0 else -1
    
    # Calculate Chaikin Money Flow (CMF)
    df['cmf'] = ChaikinMoneyFlowIndicator(high=df['high'], low=df['low'], close=df['close'], volume=df['volume'], window=28, fillna=False).chaikin_money_flow()
    cmf_value = df['cmf'].iloc[-1]
    cmf_position = 1 if cmf_value > 0 else (-1 if cmf_value < 0 else 0)
    previous_cmf_value = df['cmf'].shift(1).iloc[-1]
    cmf_cross = 1 if (cmf_value > 1 and previous_cmf_value < -1) else (-1 if (cmf_value < -1 and previous_cmf_value > 1) else 0)
        
    # Append the result to the momentum_df DataFrame using concat instead of append
    technical_long_df = pd.concat([technical_long_df, pd.DataFrame({
        'symbol': [symbol],
        'close': [df['close'].iloc[-1]],
        'vwap': [vwap],
        'pct_change': [pct_change],
        'vwap_position': [vwap_position],
        '20_50_day_ma_cross': [df['ma_cross'].iloc[-1]],
        'macd_cross': [df['macd_cross'].iloc[-1]],
        'aroon_position': [df['aroon_position'].iloc[-1]],
        'aroon_cross': [df['aroon_cross'].iloc[-1]],
        'adx': [df['adx'].iloc[-1]],
        'psar_position': [df['psar_position'].iloc[-1]],
        'psar_change': [df['psar_change'].iloc[-1]],
        'cloud_position': [df['cloud_position'].iloc[-1]],
        'ichimoku_cross': [df['ichimoku_cross'].iloc[-1]],
        'rsi': [rsi_signal],  
        'stoch': [stoch_signal],
        'atr': [df['atr'].iloc[-1]],
        'bb_position': [df['bb_position'].iloc[-1]],
        'obv_signal': [obv_signal],
        'adi_signal': [adi_signal],
        'cmf_position': [cmf_position],
        'cmf_cross': [cmf_cross],
    })], ignore_index=True)

# Order momentum_df by 'adx' from highest to lowest
technical_long_df.sort_values(by='adx', ascending=False, inplace=True)

# Set the 'symbol' column as the index of momentum_df
technical_long_df.set_index('symbol', inplace=True)

# display(trend_long_df)

### Technical Long Timeframe Report

In [ ]:
bullish_counts_long = {}
bearish_counts_long = {}

# Loop through each row in the DataFrame
for index, row in technical_long_df.iterrows():
    bullish_count = 0
    bearish_count = 0
    bullish_signals = []
    bearish_signals = []
    
    # Loop through each column to count the bullish and bearish signals
    for col in technical_long_df.columns:
        if row[col] == 1:
            bullish_count += 1
            bullish_signals.append(col)
        elif row[col] == -1:
            bearish_count += 1
            bearish_signals.append(col)
    
    # Update the counts in the dictionaries
    bullish_counts_long[index] = {
        'bullish_count': bullish_count,
        'bearish_count': bearish_count,
        'bullish_signals': bullish_signals,
        'bearish_signals': bearish_signals,
        'adx': row['adx'],
        'atr': row['atr'],
        'close': row['close'],
        'vwap': row['vwap'],
        'pct_change': row['pct_change']
        
    }
    bearish_counts_long[index] = {
        'bearish_count': bearish_count,
        'bullish_count': bullish_count,
        'bullish_signals': bullish_signals,
        'bearish_signals': bearish_signals,
        'adx': row['adx'],
        'atr': row['atr'],
        'close': row['close'],
        'vwap': row['vwap'],
        'pct_change': row['pct_change']
    }

# Filter the dictionaries to include only items with at least 5 bullish and bearish counts
filtered_bullish_long = {k: v for k, v in bullish_counts_long.items() if v['bullish_count'] >= 5}
filtered_bearish_long = {k: v for k, v in bearish_counts_long.items() if v['bearish_count'] >= 5}

# Convert to DataFrame for better visualization
df_filtered_bullish_long = pd.DataFrame.from_dict(filtered_bullish_long, orient='index')
df_filtered_bearish_long = pd.DataFrame.from_dict(filtered_bearish_long, orient='index')


### Technical Analysis on Short Time Frame

In [ ]:
technical_short_df = pd.DataFrame()

for symbol, df in ohlcv_short.items():

    pct_change = df['pct_change'].iloc[-1]
    close = df['close'].iloc[-1]
    vwap = df['vwap'].iloc[-1]
    if vwap == None:
        continue
    vwap_position = -1 if vwap < close else 1

    # Simple Moving Average Crosses
    df['20_day_sma'] = df['close'].rolling(window=20).mean()
    df['50_day_sma'] = df['close'].rolling(window=50).mean()
    df['ma_cross'] = 0  # Default value for no cross
    bullish_cross = (df['20_day_sma'] > df['50_day_sma']) & (df['20_day_sma'].shift(1) <= df['50_day_sma'].shift(1))
    df.loc[bullish_cross, 'ma_cross'] = 1
    bearish_cross = (df['20_day_sma'] < df['50_day_sma']) & (df['20_day_sma'].shift(1) >= df['50_day_sma'].shift(1))
    df.loc[bearish_cross, 'ma_cross'] = -1
    
    # MACD
    macd_indicator = MACD(df['close'])
    df['macd'] = macd_indicator.macd()
    df['macd_signal'] = macd_indicator.macd_signal()
    df['macd_diff'] = macd_indicator.macd_diff()
    df['macd_cross'] = 0 
    macd_bullish_cross = (df['macd'] > df['macd_signal']) & (df['macd'].shift(1) <= df['macd_signal'].shift(1))
    df.loc[macd_bullish_cross, 'macd_cross'] = 1
    macd_bearish_cross = (df['macd'] < df['macd_signal']) & (df['macd'].shift(1) >= df['macd_signal'].shift(1))
    df.loc[macd_bearish_cross, 'macd_cross'] = -1

    # Aroon Indicator
    aroon_indicator = AroonIndicator(high=df['high'], low=df['low'], window=25)
    df['aroon_up'] = aroon_indicator.aroon_up()
    df['aroon_down'] = aroon_indicator.aroon_down()
    df['aroon_cross'] = 0
    df['aroon_position'] = 0
    aroon_position_condition_1 = (df['aroon_up'] > 80) & (df['aroon_down'] < 20)
    df.loc[aroon_position_condition_1, 'aroon_position'] = 1
    aroon_position_condition_2 = (df['aroon_down'] > 80) & (df['aroon_up'] < 20)
    df.loc[aroon_position_condition_2, 'aroon_position'] = -1
    aroon_bull_cross = (df['aroon_up'] > df['aroon_down']) & (df['aroon_up'].shift(1) <= df['aroon_down'].shift(1))
    df.loc[aroon_bull_cross, 'aroon_cross'] = 1
    aroon_bear_cross = (df['aroon_up'] < df['aroon_down']) & (df['aroon_up'].shift(1) >= df['aroon_down'].shift(1))
    df.loc[aroon_bear_cross, 'aroon_cross'] = -1

    # ADX
    adx_indicator = ADXIndicator(high=df['high'], low=df['low'], close=df['close'], window=14, fillna=False)
    df['adx'] = adx_indicator.adx()

    # Parabolic (SAR)
    psar_indicator = PSARIndicator(df['high'], df['low'], df['close'], step=0.02, max_step=0.2)
    df['psar'] = psar_indicator.psar()
    df['psar_position'] = 0
    df.loc[df['psar'] > df['close'], 'psar_position'] = -1
    df.loc[df['psar'] < df['close'], 'psar_position'] = 1
    # When there is a change in the dot. Above to below is bullish (1), below to above is bearish (-1), and 0 otherwise
    df['psar_change'] = 0  # Default value for no change
    psar_above_to_below = (df['psar_position'] > 0) & (df['psar_position'].shift(1) <= 0)
    psar_below_to_above = (df['psar_position'] < 0) & (df['psar_position'].shift(1) >= 0)
    df.loc[psar_above_to_below, 'psar_change'] = 1  # Bullish change from above to below
    df.loc[psar_below_to_above, 'psar_change'] = -1  # Bearish change from below to above

    # Ichimoku Cloud
    ichimoku = IchimokuIndicator(high=df['high'], low=df['low'], window1=9, window2=26, window3=52, visual=False)
    df['conversion_line'] = ichimoku.ichimoku_conversion_line()
    df['base_line'] = ichimoku.ichimoku_base_line()
    df['leading_span_a'] = ichimoku.ichimoku_a()
    df['leading_span_b'] = ichimoku.ichimoku_b()
    # Identify bullish and bearish crosses
    bullish_cross = (df['conversion_line'] > df['base_line']) & (df['conversion_line'].shift(1) <= df['base_line'].shift(1))
    bearish_cross = (df['conversion_line'] < df['base_line']) & (df['conversion_line'].shift(1) >= df['base_line'].shift(1))
    df['ichimoku_cross'] = 0
    df.loc[bullish_cross, 'ichimoku_cross'] = 1
    df.loc[bearish_cross, 'ichimoku_cross'] = -1
    # Determine the position of the price relative to the cloud
    above_cloud = (df['close'] > df[['leading_span_a', 'leading_span_b']].max(axis=1))
    below_cloud = (df['close'] < df[['leading_span_a', 'leading_span_b']].min(axis=1))
    # Corrected logic for determining if the price is within the cloud
    in_cloud = ~above_cloud & ~below_cloud
    df['cloud_position'] = 0
    df.loc[above_cloud, 'cloud_position'] = 1
    df.loc[below_cloud, 'cloud_position'] = -1
    df.loc[in_cloud, 'cloud_position'] = 0

    # RSI
    rsi_period = 14 
    rsi_indicator = RSIIndicator(df['close'], rsi_period)
    df['rsi'] = rsi_indicator.rsi()
    rsi_value = df['rsi'].iloc[-1]
    rsi_signal = 1 if rsi_value < 30 else -1 if rsi_value > 70 else 0

    # Stochastic Oscillator
    stoch_window = 14
    stoch_smooth_window = 3
    stoch = StochasticOscillator(high=df['high'], low=df['low'], close=df['close'], window=stoch_window, smooth_window=stoch_smooth_window)
    df['%K'] = stoch.stoch()
    df['%D'] = stoch.stoch_signal()
    stoch_signal = 1 if df['%K'].iloc[-1] < 20 else -1 if df['%K'].iloc[-1] > 80 else 0

    # Average True Range
    atr_indicator = AverageTrueRange(df['high'], df['low'], df['close'], window=14)
    df['atr'] = atr_indicator.average_true_range()

    # Bollinger Bands
    indicator_bb = BollingerBands(close=df['close'], window=20, window_dev=2)
    df['bb_mavg'], df['bb_hband'], df['bb_lband'] = indicator_bb.bollinger_mavg(), indicator_bb.bollinger_hband(), indicator_bb.bollinger_lband()
    df['bb_position'] = 0
    df['bb_position'] = np.where(df['close'] > df['bb_hband'], -1, np.where(df['close'] < df['bb_lband'], 1, 0))

    # Calculate On Balance Volume (OBV)
    obv_indicator = OnBalanceVolumeIndicator(df['close'], df['volume'])
    df['obv'] = obv_indicator.on_balance_volume()
    obv_trend = df['obv'].diff(periods=5)
    obv_signal = 1 if obv_trend.iloc[-1] > 0 else -1

    adi_indicator = AccDistIndexIndicator(high=df['high'], low=df['low'], close=df['close'], volume=df['volume'])
    df['adi'] = adi_indicator.acc_dist_index()
    adi_trend = df['adi'].diff(periods=5)
    adi_signal = 1 if adi_trend.iloc[-1] > 0 else -1
    
    # Calculate Chaikin Money Flow (CMF)
    df['cmf'] = ChaikinMoneyFlowIndicator(high=df['high'], low=df['low'], close=df['close'], volume=df['volume'], window=28, fillna=False).chaikin_money_flow()
    cmf_value = df['cmf'].iloc[-1]
    cmf_position = 1 if cmf_value > 0 else (-1 if cmf_value < 0 else 0)
    previous_cmf_value = df['cmf'].shift(1).iloc[-1]
    cmf_cross = 1 if (cmf_value > 1 and previous_cmf_value < -1) else (-1 if (cmf_value < -1 and previous_cmf_value > 1) else 0)
        
    # Append the result to the momentum_df DataFrame using concat instead of append
    technical_short_df = pd.concat([technical_short_df, pd.DataFrame({
        'symbol': [symbol],
        'close': [df['close'].iloc[-1]],
        'vwap': [vwap],
        'pct_change': [pct_change],
        'vwap_position': [vwap_position],
        '20_50_day_ma_cross': [df['ma_cross'].iloc[-1]],
        'macd_cross': [df['macd_cross'].iloc[-1]],
        'aroon_position': [df['aroon_position'].iloc[-1]],
        'aroon_cross': [df['aroon_cross'].iloc[-1]],
        'adx': [df['adx'].iloc[-1]],
        'psar_position': [df['psar_position'].iloc[-1]],
        'psar_change': [df['psar_change'].iloc[-1]],
        'cloud_position': [df['cloud_position'].iloc[-1]],
        'ichimoku_cross': [df['ichimoku_cross'].iloc[-1]],
        'rsi': [rsi_signal],  
        'stoch': [stoch_signal],
        'atr': [df['atr'].iloc[-1]],
        'bb_position': [df['bb_position'].iloc[-1]],
        'obv_signal': [obv_signal],
        'adi_signal': [adi_signal],
        'cmf_position': [cmf_position],
        'cmf_cross': [cmf_cross],
    })], ignore_index=True)

# Order momentum_df by 'adx' from highest to lowest
technical_short_df.sort_values(by='adx', ascending=False, inplace=True)

# Set the 'symbol' column as the index of momentum_df
technical_short_df.set_index('symbol', inplace=True)

# display(trend_short_df)

### Technical Report on Short Timeframe

In [ ]:
bullish_counts_short = {}
bearish_counts_short = {}

# Loop through each row in the DataFrame
for index, row in technical_short_df.iterrows():
    bullish_count = 0
    bearish_count = 0
    bullish_signals = []
    bearish_signals = []
    
    # Loop through each column to count the bullish and bearish signals
    for col in technical_short_df.columns:
        if row[col] == 1:
            bullish_count += 1
            bullish_signals.append(col)
        elif row[col] == -1:
            bearish_count += 1
            bearish_signals.append(col)
    
    # Update the counts in the dictionaries
    bullish_counts_short[index] = {
        'bullish_count': bullish_count,
        'bearish_count': bearish_count,
        'bullish_signals': bullish_signals,
        'bearish_signals': bearish_signals,
        'adx': row['adx'],
        'atr': row['atr'],
        'close': row['close'],
        'vwap': row['vwap'],
        'pct_change': row['pct_change']
    }
    bearish_counts_short[index] = {
        'bearish_count': bearish_count,
        'bullish_count': bullish_count,
        'bullish_signals': bullish_signals,
        'bearish_signals': bearish_signals,
        'adx': row['adx'],
        'atr': row['atr'],
        'close': row['close'],
        'vwap': row['vwap'],
        'pct_change': row['pct_change']
    }

# Filter the dictionaries to include only items with at least 3 bullish and bearish counts
filtered_bullish_short = {k: v for k, v in bullish_counts_short.items() if v['bullish_count'] >= 5}
filtered_bearish_short = {k: v for k, v in bearish_counts_short.items() if v['bearish_count'] >= 5}

# Convert to DataFrame for better visualization
df_filtered_bullish_short = pd.DataFrame.from_dict(filtered_bullish_short, orient='index')
df_filtered_bearish_short = pd.DataFrame.from_dict(filtered_bearish_short, orient='index')

## Output Reports

In [ ]:
print('===================== Daily Timeframe Technical Analysis ===============================')
print('=======================================================================================')
print("Filtered Bullish Symbols with at least 5 bullish counts:")
print(tabulate(df_filtered_bullish_daily, headers='keys', tablefmt='psql', showindex=True))
print("\nFiltered Bearish Symbols with at least 5 bearish counts:")
print(tabulate(df_filtered_bearish_daily, headers='keys', tablefmt='psql', showindex=True))
print('\n\n')

print('===================== Long Timeframe Technical Analysis ===============================')
print('=======================================================================================')
print("Filtered Bullish Symbols with at least 5 bullish counts:")
print(tabulate(df_filtered_bullish_long, headers='keys', tablefmt='psql', showindex=True))
print("\nFiltered Bearish Symbols with at least 5 bearish counts:")
print(tabulate(df_filtered_bearish_long, headers='keys', tablefmt='psql', showindex=True))
print('\n\n')

print('===================== Short Timeframe Technical Analysis ===============================')
print('========================================================================================')
print("Filtered Bullish Symbols with at least 5 bullish counts:")
print(tabulate(df_filtered_bullish_short, headers='keys', tablefmt='psql', showindex=True))
print("\nFiltered Bearish Symbols with at least 5 bearish counts:")
print(tabulate(df_filtered_bearish_short, headers='keys', tablefmt='psql', showindex=True))
print('\n\n')

bullish_daily_symbols = df_filtered_bullish_daily.index.values.tolist()
bearish_daily_symbols = df_filtered_bearish_daily.index.values.tolist()

bullish_long_symbols = df_filtered_bullish_long.index.values.tolist()
bearish_long_symbols = df_filtered_bearish_long.index.values.tolist()

bullish_short_symbols = df_filtered_bullish_short.index.values.tolist()
bearish_short_symbols = df_filtered_bearish_short.index.values.tolist()

# Convert lists to sets
set_long = set(bullish_long_symbols)
set_daily = set(bullish_daily_symbols)
set_short = set(bullish_short_symbols)

# Find common elements
common_symbols = list(set_long & set_daily & set_short)
print(common_symbols)


